## 题解

### mlp

In [ ]:
# 导入所需的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
print("done")

- 先查看数据的基本情况,检查缺失

In [ ]:
# 读取训练集和测试集
train_df = pd.read_csv('E:\\download\\AI派2025招新第一轮测试\\AI派2025招新第一轮测试\\MBAAdmission\\train.csv')  
test_df = pd.read_csv('E:\\download\\AI派2025招新第一轮测试\\AI派2025招新第一轮测试\\MBAAdmission\\test.csv')

print(train_df.head())
# print(train_df.tail())

print(f"训练集形状: {train_df.shape}")  #
print(f"测试集形状: {test_df.shape}")  #

print("特征列名: ", train_df.columns.tolist()) #

print(train_df.info())  # info方法查看类型和缺失值
print(test_df.info())


- 先对已有数据求出application_id,gpa,gmat,work_exp基本的统计信息,方便填入缺失的信息.
类别型特征的缺失值常用众数填充

In [ ]:
print(train_df.isnull().sum())
# 例如：race 列可能因国际学生无数据而出现缺失
# # 对数值列（如 gpa、gmat、work_exp）进行统计描述
print(train_df.describe())  # 包含均值、标准差、最小值、最大值、分位数等

non_num_cols = train_df.select_dtypes(include=['object'])
#non_num_cols_test = test_df.select_dtypes(include=['object'])

print(non_num_cols.count())
non_num_cols = non_num_cols.columns.tolist()
#non_num_cols_ = non_num_cols_.colums.tolist()

for col in non_num_cols:
    print(f"===== 特征：{col} =====")
    
    # 获取唯一类别（含缺失值可保留）
    unique_categories = train_df[col].unique()
    print(f"唯一类别（共{len(unique_categories)}种）：{unique_categories}")
    #temp["唯一类别数"].append(len(unique_categories))
    
    # 获取每个类别的出现次数（排除缺失值，按降序排列）
    value_counts = train_df[col].value_counts(dropna=False)  # dropna=False包含缺失值
    #temp["类别"].append(value_counts.keys().tolist())
    print("类别出现次数：")
    print(value_counts)
    print("\n" + "-"*50 + "\n")  # 分隔线
# 按逻辑填补缺失值
train_df['gender'] = train_df['gender'].fillna(train_df['gender'].mode()[0])
train_df['international'] = train_df['international'].fillna(train_df['international'].mode()[0])
train_df['gpa'] = train_df['gpa'].fillna(train_df['gpa'].mean())
train_df['major'] = train_df['major'].fillna(train_df['major'].mode()[0])
train_df['race'] = train_df['race'].fillna(train_df['race'].mode()[0])
train_df['gmat'] = train_df['gmat'].fillna(train_df['gmat'].median())
train_df['work_exp'] = train_df['work_exp'].fillna(0)
train_df['work_industry'] = train_df['work_industry'].fillna('other')

test_df['gender'] = test_df['gender'].fillna(test_df['gender'].mode()[0])
test_df['international'] = test_df['international'].fillna(test_df['international'].mode()[0])
test_df['gpa'] = test_df['gpa'].fillna(test_df['gpa'].mean())
test_df['major'] = test_df['major'].fillna(test_df['major'].mode()[0])
test_df['race'] = test_df['race'].fillna(test_df['race'].mode()[0])
test_df['gmat'] = test_df['gmat'].fillna(test_df['gmat'].median())
test_df['work_exp'] = test_df['work_exp'].fillna(0)
test_df['work_industry'] = test_df['work_industry'].fillna('other')
print(train_df.isnull().sum())


# 注：mode() 返回 Series，取 [0] 获取第一个众数

In [ ]:
# 对分类特征进行one-hot编码
# 首先确定需要编码的分类列（通常是object类型的列）
categorical_cols = train_df.select_dtypes(include=['object']).columns.tolist()
categorical_cols_ = test_df.select_dtypes(include=['object']).columns.tolist()
# gender he international 不用onehot
exclude_cols = ['international','gender','admission']

categorical_cols = [col for col in categorical_cols if col not in exclude_cols]
categorical_cols_ = [col for col in categorical_cols_ if col not in exclude_cols]
print(f"将对以下分类列进行one-hot编码: {categorical_cols}")

# 进行one-hot编码
print(train_df.head(1))
train_df_encoded = pd.get_dummies(train_df, columns=categorical_cols, drop_first=True)
train_df_encoded['international'] = train_df['international'].map({True: 1, False: 0})#对international进行二进制编码
train_df_encoded['gender'] = train_df['gender'].map({'Male': 1, 'Female': 0})#对gender进行标签编码
train_df_encoded['admission'] = train_df['admission'].map({'Reject': 1, 'Waitlist': 0,'Admit':2})#对admission进行标签编码

test_df_encoded = pd.get_dummies(test_df, columns=categorical_cols, drop_first=True)
test_df_encoded['international'] = test_df['international'].map({True: 1, False: 0})#对international进行二进制编码
test_df_encoded['gender'] = test_df['gender'].map({'Male': 1, 'Female': 0})#对gender进行标签编码
test_df_encoded['admission'] = test_df['admission'].map({'Reject': 1, 'Waitlist': 0,'Admit':2})#对admission进行标签编码
for col in train_df_encoded:
    if col not in test_df_encoded.columns:
        print("===")
        print(col)
        test_df_encoded[col] = 0
# 查看编码后的数据集形状
print(f"编码前数据集形状: {train_df.shape}")
print(f"编码后数据集形状: {train_df_encoded.shape}")
print(f"编码前数据集形状2: {test_df.shape}")
print(f"编码后数据集形状2: {test_df_encoded.shape}")

# 查看前几行编码结果
print("\n编码后的数据集前5行:")
print(train_df_encoded.head(1))
print("\n编码后的数据集前5行:")
print(test_df_encoded.head(1))


对数值取值差异大的连续型gpa,gmat,work_exp进行归一化

In [ ]:

# 数据标准化
train_df_encoded['gpa'] = (train_df['gpa'] - train_df['gpa'].mean()) / train_df['gpa'].std()
train_df_encoded['gmat'] = (train_df['gmat'] - train_df['gmat'].mean()) / train_df['gmat'].std()
train_df_encoded['work_exp'] = (train_df['work_exp'] - train_df['work_exp'].mean()) / train_df['work_exp'].std()

# 数据标准化_test
test_df_encoded['gpa'] = (test_df['gpa'] - test_df['gpa'].mean()) / test_df['gpa'].std()
test_df_encoded['gmat'] = (test_df['gmat'] - test_df['gmat'].mean()) / test_df['gmat'].std()
test_df_encoded['work_exp'] = (test_df['work_exp'] - test_df['work_exp'].mean()) / test_df['work_exp'].std()
print(test_df_encoded.head(1))


In [ ]:
import numpy as np
import scipy as sp
import sklearn as sk
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier

lr = 0.0001
epoch = 50
batch = 64
# 4. 使用scikit-learn的MLP分类器
print("\n训练MLP分类器...")
#x_train = np.array(train_df.drop(columns=['admission']))

# train_df_encoded['admission_status'] = 0  # 默认拒绝
# train_df_encoded.loc[train_df_encoded['admission_Waitlist'] == True, 'admission_status'] = 1
# train_df_encoded.loc[(train_df_encoded['admission_Reject'] == False) & (train_df_encoded['admission_Waitlist'] == False), 'admission_status'] = 2

for col in train_df_encoded.columns:
    if train_df_encoded[col].dtype == bool:
        train_df_encoded[col] = train_df_encoded[col].map({True: 1, False: 0})

for col in test_df_encoded.columns:
    if test_df_encoded[col].dtype == bool:
        test_df_encoded[col] = test_df_encoded[col].map({True: 1, False: 0})
# 3. 选择特征列（排除ID、目标变量相关列）
feature_cols = [col for col in train_df_encoded.columns if col not in [
    'application_id','admission'
]]
feature_cols_ = [col for col in test_df_encoded.columns if col not in [
    'application_id','admission'
]]
#print(train_df_encoded.columns)
# 4. 转换为NumPy数组
X_train = train_df_encoded[feature_cols].to_numpy()  # 特征数据
y_train = train_df_encoded[['admission']].to_numpy()  # 目标变量

X_test = test_df_encoded[feature_cols_].to_numpy()  # 特征数据
y_test = test_df_encoded[['admission']].to_numpy()  # 目标变量
print("特征数据形状:", X_train.shape)
print("目标变量形状:", y_train.shape)
print("\n前5行特征数据:")
print(X_train[:5])
print("\n前5行目标变量:")
print(y_train[:5])
# hidden_dim
# [input_dim, 128], ReLU()
# [128, 256], ReLU()
# [256, output_dim]
# 定义MLP模型，使用指定的隐藏层结构

mlp = MLPClassifier(
    hidden_layer_sizes=(128, 256),  # 第一层128个神经元，第二层256个神经元
    activation='relu',              # 使用ReLU激活函数，符合要求
    solver='adam',                  # Adam优化器，适合大多数场景
    alpha=0.001,                   # L2正则化参数
    batch_size=batch,              # 自动决定批次大小
    learning_rate='constant',       # 固定学习率
    learning_rate_init=lr,       # 初始学习率
    max_iter=epoch,                   # 最大迭代次数
    shuffle=True,                   # 每次迭代前打乱数据
    random_state=42,                # 随机种子，保证结果可复现
    verbose=True,                   # 训练过程中打印日志
    early_stopping=True,            # 启用早停策略
    validation_fraction=0.1,        # 10%的训练数据作为验证集
    tol=1e-4,                       # 早停的容差
    n_iter_no_change=10             # 10轮迭代无改善则停止
)
X_train_for_torch = X_train
y_train_for_torch = y_train
mlp.fit(X_train, y_train)

# 5. 模型评估
print("\n模型评估:")
y_pred = mlp.predict(X_test)
y_pred_proba = mlp.predict_proba(X_test)[:, 1]  # 正类的预测概率

print(f"测试集准确率: {accuracy_score(y_test, y_pred):.4f}")
print("\n分类报告:")
print(classification_report(y_test, y_pred))
# print("\n混淆矩阵:")
# print(confusion_matrix(y_test, y_pred))

# 6. 绘制训练损失曲线
plt.figure(figsize=(10, 6))
plt.plot(mlp.loss_curve_)
plt.title('MLP Training Loss Curve')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.grid(True)
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

class MLP(nn.Module):
    def __init__(self, input_dim, num_classes, dropout_rate=0.2):
        # 在初始化模型之前添加
        print(torch.cuda.is_available()) 
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(self.device)
        super().__init__()
        self.linear1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(128, 256)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.linear3 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.linear3(x)
        return x
    
    def predict(self, x):
        x = x.to(self.device)
        with torch.no_grad():
            outputs = self.forward(x)
            return torch.argmax(outputs, dim=1)
    
    def fit(self, X_train, y_train, X_val=None, y_val=None, epochs=100, lr=0.001, batch_size=32):
        """
        训练模型
        
        参数:
        X_train: 训练特征，形状为 [样本数, 特征数]
        y_train: 训练标签，形状为 [样本数]，每个元素是0,1,2之类的类别索引
        """
        # 确保标签是长整型
        if not isinstance(y_train, torch.Tensor):
            y_train = torch.tensor(y_train, dtype=torch.long)
        elif y_train.dtype != torch.long:
            y_train = y_train.long()
        
        optimizer = optim.Adam(self.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()
        
        # 创建数据加载器
        dataset = torch.utils.data.TensorDataset(X_train, y_train)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        
        for epoch in range(epochs):
            total_loss = 0
            self.train()  # 设置为训练模式
            
            for batch_X, batch_y in dataloader:
                batch_X, batch_y = batch_X.to(self.device), batch_y.to(self.device)
                optimizer.zero_grad()
                outputs = self.forward(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            
            avg_loss = total_loss / len(dataloader)
            
            if (epoch + 1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')
                
                # 验证集评估
                if X_val is not None and y_val is not None:
                    val_accuracy = self.evaluate(X_val, y_val)
                    print(f'Validation Accuracy: {val_accuracy:.4f}')
    
    def evaluate(self, X, y):
        """评估模型性能"""
        # 在evaluate方法中：
        X, y = X.to(self.device), y.to(self.device)
        self.eval()  # 设置为评估模式
        with torch.no_grad():
            # 预测结果需要计算指标时：
            predictions = self.predict(X)
            predictions = predictions.cpu().numpy()
            y_true = y.cpu().numpy()
            accuracy = accuracy_score(y_true, predictions)
        return accuracy

# 使用示例
# 确保您的数据格式正确：
# X_train_for_torch: 形状为 [样本数, 特征数] 的numpy数组或tensor
# y_train_for_torch: 形状为 [样本数] 的类别索引数组 (如 [0, 1, 2, 0, 1, 2,...])



X_train = torch.tensor(X_train_for_torch, dtype=torch.float32)
y_train = torch.tensor(y_train_for_torch, dtype=torch.long)
y_train = y_train.squeeze()

# cuda = torch.device('cuda')
# X_train = X_train.to(cuda)
# y_train = y_train.to(cuda)

# 检查标签格式
print(f"标签形状: {y_train.shape}")
print(f"标签唯一值: {torch.unique(y_train)}")
print(f"标签范围: {y_train.min()} 到 {y_train.max()}")

input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))
print(f"输入维度: {input_dim}, 类别数: {num_classes}")

mlp = MLP(input_dim, num_classes)
mlp = mlp.to(mlp.device)  # 将模型移动到GPU
mlp.fit(X_train, y_train,epochs = epoch,lr = lr)

# 预测
X_test = torch.tensor(X_test, dtype=torch.float32)
y_pred = mlp.predict(X_test)
y_pred = y_pred.cpu().numpy()

acr =0
for i ,j in zip(y_test,y_pred):
    if i == j:
        acr+=1
    else:
        pass
acr = acr/len(y_test)
print(f'准确率为：{acr}')


In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib.font_manager as fm
# from matplotlib.ticker import PercentFormatter
# import warnings

# # -------------------------- 1. 字体设置（彻底解决中文显示问题）--------------------------
# # 强制指定支持完整中文的字体列表，按优先级排序
# plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC", "Microsoft YaHei", "Arial Unicode MS"]
# plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# # 验证字体是否有效
# def verify_chinese_font():
#     try:
#         fig = plt.figure()
#         ax = fig.add_subplot(111)
#         ax.text(0.5, 0.5, '测试中文显示：录取、沃顿商学院、均值', 
#                 ha='center', va='center', fontsize=12)
#         plt.close(fig)
#         return True
#     except:
#         return False

# # 如果字体验证失败，尝试更通用的配置
# if not verify_chinese_font():
#     plt.rcParams["font.family"] = ["Arial Unicode MS", "sans-serif"]

# plt.rcParams['font.size'] = 10
# plt.rcParams['figure.dpi'] = 100
# plt.style.use('seaborn-v0_8-whitegrid')

# # 定义沃顿风格配色
# colors = ['#2E86AB', '#A23B72', '#F18F01']  # 沃顿蓝、紫色、橙色


# # -------------------------- 2. 核心可视化函数（保持不变）--------------------------
# def plot_wharton_admission_overview(df):
#     """1. 沃顿申请整体概览：录取率+关键结构特征"""
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
#     # 子图1：录取状态分布
#     admission_count = df['admission'].value_counts()
#     admission_pct = df['admission'].value_counts(normalize=True) * 100
    
#     wedges, texts, autotexts = ax1.pie(
#         admission_count.values, 
#         labels=admission_count.index,
#         autopct='%1.1f%%',
#         colors=colors,
#         startangle=90,
#         textprops={'fontsize': 11}
#     )
#     ax1.set_title('沃顿商学院2025届申请录取状态分布\n（总申请数：{}）'.format(len(df)), fontsize=13, pad=20)
    
#     # 子图2：核心结构特征汇总
#     gender_pct = df['gender'].value_counts(normalize=True) * 100
#     international_pct = df['international'].value_counts(normalize=True) * 100
#     major_pct = df['major'].value_counts(normalize=True) * 100
    
#     summary_text = f"""
#     申请者核心结构：
#     • 性别比例：男性 {gender_pct.get('男', 0):.1f}% / 女性 {gender_pct.get('女', 0):.1f}%
#     • 国际学生占比：{international_pct.get('是', 0):.1f}%
#     • 本科专业分布：
#       - 商科 {major_pct.get('商科', 0):.1f}%
#       - 理工科 {major_pct.get('理工科', 0):.1f}%
#       - 人文学科 {major_pct.get('人文学科', 0):.1f}%
#     """
#     ax2.text(0.1, 0.9, summary_text, transform=ax2.transAxes, 
#              fontsize=12, verticalalignment='top',
#              bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))
#     ax2.axis('off')
#     ax2.set_title('申请者核心结构概览', fontsize=13, pad=20)
    
#     plt.tight_layout()
#     plt.show()


# def plot_academic_indicators(df):
#     """2. 学术硬指标分析：GPA/GMAT分布+与录取的关系"""
#     fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
#     # 子图1：GPA整体分布
#     sns.histplot(df['gpa'], kde=True, bins=20, color='#2E86AB', ax=ax1)
#     ax1.axvline(df['gpa'].mean(), color='red', linestyle='--', linewidth=2, 
#                 label=f'均值：{df["gpa"].mean():.2f}')
#     ax1.set_title('申请者GPA分布（4.0分制）', fontsize=12)
#     ax1.set_xlabel('GPA')
#     ax1.set_ylabel('申请人数')
#     ax1.legend()
    
#     # 子图2：GMAT整体分布
#     sns.histplot(df['gmat'], kde=True, bins=20, color='#A23B72', ax=ax2)
#     ax2.axvline(df['gmat'].mean(), color='red', linestyle='--', linewidth=2, 
#                 label=f'均值：{df["gmat"].mean():.0f}')
#     ax2.set_title('申请者GMAT分布（满分800）', fontsize=12)
#     ax2.set_xlabel('GMAT分数')
#     ax2.set_ylabel('申请人数')
#     ax2.legend()
    
#     # 子图3：不同录取状态的GPA箱线图
#     sns.boxplot(x='admission', y='gpa', data=df, hue='admission',
#                 palette=colors, ax=ax3, legend=False)
#     ax3.set_title('不同录取状态的GPA分布', fontsize=12)
#     ax3.set_xlabel('录取状态')
#     ax3.set_ylabel('GPA')
#     # 添加各组均值标注
#     for i, status in enumerate(df['admission'].unique()):
#         mean_gpa = df[df['admission'] == status]['gpa'].mean()
#         ax3.text(i, mean_gpa + 0.05, f'均值：{mean_gpa:.2f}', ha='center', fontsize=10)
    
#     # 子图4：不同录取状态的GMAT箱线图
#     sns.boxplot(x='admission', y='gmat', data=df, hue='admission',
#                 palette=colors, ax=ax4, legend=False)
#     ax4.set_title('不同录取状态的GMAT分布', fontsize=12)
#     ax4.set_xlabel('录取状态')
#     ax4.set_ylabel('GMAT分数')
#     # 添加各组均值标注
#     for i, status in enumerate(df['admission'].unique()):
#         mean_gmat = df[df['admission'] == status]['gmat'].mean()
#         ax4.text(i, mean_gmat + 20, f'均值：{mean_gmat:.0f}', ha='center', fontsize=10)
    
#     plt.tight_layout()
#     plt.show()


# # -------------------------- 3. 其他函数和主程序（保持不变）--------------------------
# def plot_work_background(df):
#     """3. 工作背景分析：行业分布+工作年限与录取的关系"""
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
    
#     # 子图1：工作行业分布（Top8行业）
#     work_industry_count = df['work_industry'].value_counts()
#     top8_industries = work_industry_count.head(8)
#     other_count = work_industry_count.iloc[8:].sum()
#     if other_count > 0:
#         top8_industries['其他'] = other_count
    
#     sns.barplot(x=top8_industries.values, y=top8_industries.index, color='#F18F01', ax=ax1)
#     # 添加数量+占比标注
#     total = len(df)
#     for i, (industry, count) in enumerate(top8_industries.items()):
#         pct = (count / total) * 100
#         ax1.text(count + 5, i, f'{count}人（{pct:.1f}%）', va='center', fontsize=9)
#     ax1.set_title('申请者工作行业分布（Top8）', fontsize=12)
#     ax1.set_xlabel('申请人数')
#     ax1.set_ylabel('工作行业')
    
#     # 子图2：工作年限与录取率的关系
#     df['work_exp_group'] = pd.cut(
#         df['work_exp'], 
#         bins=[0, 1, 3, 5, np.inf], 
#         labels=['0-1年', '1-3年', '3-5年', '5年以上']
#     )
#     # 计算每组的录取率
#     work_exp_admission = df.groupby('work_exp_group')['admission'].apply(
#         lambda x: (x == '录取').sum() / len(x) * 100
#     ).reset_index()
    
#     sns.barplot(x='work_exp_group', y='admission', data=work_exp_admission, 
#                 hue='work_exp_group', color='#2E86AB', ax=ax2, legend=False)
#     ax2.yaxis.set_major_formatter(PercentFormatter())
#     # 添加录取率标注
#     for i, rate in enumerate(work_exp_admission['admission']):
#         ax2.text(i, rate + 1, f'{rate:.1f}%', ha='center', fontsize=10)
#     ax2.set_title('不同工作年限的录取率', fontsize=12)
#     ax2.set_xlabel('工作年限')
#     ax2.set_ylabel('录取率（%）')
    
#     plt.tight_layout()
#     plt.show()


# def plot_key_correlations(df):
#     """4. 关键特征相关性：学术+工作特征热力图+国际学生录取差异"""
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
#     # 子图1：数值特征相关性热力图
#     numerical_cols = ['gpa', 'gmat', 'work_exp']
#     corr_matrix = df[numerical_cols].corr()
    
#     sns.heatmap(
#         corr_matrix, 
#         annot=True, 
#         fmt='.2f', 
#         cmap='coolwarm', 
#         square=True, 
#         cbar_kws={'shrink': 0.8},
#         ax=ax1
#     )
#     ax1.set_title('学术+工作特征相关性热力图', fontsize=12)
    
#     # 子图2：国际学生vs本地学生的录取率对比
#     international_admission = df.groupby('international')['admission'].apply(
#         lambda x: (x == '录取').sum() / len(x) * 100
#     ).reset_index()
    
#     sns.barplot(x='international', y='admission', data=international_admission, 
#                 hue='international', palette=['#A23B72', '#2E86AB'], ax=ax2, legend=False)
#     ax2.yaxis.set_major_formatter(PercentFormatter())
#     # 添加录取率标注
#     for i, rate in enumerate(international_admission['admission']):
#         ax2.text(i, rate + 1, f'{rate:.1f}%', ha='center', fontsize=10)
#     ax2.set_title('国际学生vs本地学生录取率对比', fontsize=12)
#     ax2.set_xlabel('是否国际学生')
#     ax2.set_ylabel('录取率（%）')
    
#     plt.tight_layout()
#     plt.show()


# def plot_major_industry_admission(df):
#     """5. 本科专业与工作行业的录取率分析"""
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))
    
#     # 子图1：不同本科专业的录取率
#     major_admission = df.groupby('major')['admission'].apply(
#         lambda x: (x == '录取').sum() / len(x) * 100
#     ).reset_index()
    
#     sns.barplot(x='major', y='admission', data=major_admission, 
#                 hue='major', palette=colors, ax=ax1, legend=False)
#     ax1.yaxis.set_major_formatter(PercentFormatter())
#     for i, rate in enumerate(major_admission['admission']):
#         ax1.text(i, rate + 1, f'{rate:.1f}%', ha='center', fontsize=10)
#     ax1.set_title('不同本科专业的录取率', fontsize=12)
#     ax1.set_xlabel('本科专业')
#     ax1.set_ylabel('录取率（%）')
    
#     # 子图2：Top5工作行业的录取率
#     top5_industries = df['work_industry'].value_counts().head(5).index
#     industry_admission = df[df['work_industry'].isin(top5_industries)].groupby('work_industry')['admission'].apply(
#         lambda x: (x == '录取').sum() / len(x) * 100
#     ).reset_index().sort_values('admission', ascending=False)
    
#     sns.barplot(x='admission', y='work_industry', data=industry_admission, 
#                 hue='work_industry', palette=colors, ax=ax2, legend=False)
#     ax2.xaxis.set_major_formatter(PercentFormatter())
#     for i, rate in enumerate(industry_admission['admission']):
#         ax2.text(rate + 1, i, f'{rate:.1f}%', va='center', fontsize=10)
#     ax2.set_title('Top5工作行业的录取率（从高到低）', fontsize=12)
#     ax2.set_xlabel('录取率（%）')
#     ax2.set_ylabel('工作行业')
    
#     plt.tight_layout()
#     plt.show()


# warnings.filterwarnings("ignore", category=UserWarning)
# # 生成模拟数据
# np.random.seed(42)
# n_samples = 3200

# train_df_encoded =train_df
# # 执行分析
# print("沃顿商学院2025届申请数据概况：")
# print(f"总申请人数：{len(train_df_encoded)}人")
# print(f"录取率：{(train_df_encoded['admission'] == '录取').mean():.2%}")
# print(f"等待名单比例：{(train_df_encoded['admission'] == '等待名单').mean():.2%}")
# print("\n开始可视化分析...\n")

# plot_wharton_admission_overview(train_df_encoded)
# plot_academic_indicators(train_df_encoded)
# plot_work_background(train_df_encoded)
# plot_key_correlations(train_df_encoded)
# plot_major_industry_admission(train_df_encoded)

# print("可视化分析完成！")


### CNN

In [ ]:
import numpy as np

def linear_layer(x, w, b):
    """
    线性层前向传播
    x: 输入张量，形状为 (N, C, H, W)
    w: 权重张量，形状为 (C, H, W)
    b: 偏置张量，形状为 (1, 1, 1)
    """
    return x @ w + b

def relu(x):
    """
    对输入张量 x 执行元素级的 ReLU (Rectified Linear Unit) 操作。
    公式为: f(x) = max(0, x)
    """
    # ===== 在此实现 =====
    return np.maximum(0, x)

def flatten(x):
    """
    将一个四维张量 (N, C, H, W) 展平为一个二维张量 (N, C*H*W)。
    N 是批量大小，需要保持不变。
    """
    # ===== 在此实现 =====
    return x.reshape(x.shape[0], -1)


In [ ]:
x = np.array([[-2], [-1], [0], [1], [2]])
w1, b1 = np.array([[2]]),np.array([-1])
w2, b2 = np.array([[-1]]),np.array([0.5])
def forword_2_layer(x,use_relu = False):
    """
    2层神经网络前向传播
    x: 输入张量，形状为 (N, 1)
    """
    z1 = linear_layer(x, w1, b1)
    if use_relu:
        a1 = relu(z1)
    else:
        a1 = z1
    z2 = linear_layer(a1, w2, b2)
    return z2
res1 = forword_2_layer(x)
res2 = forword_2_layer(x,use_relu = True)
print(res1)
print(res2)


ReLU函数将负输入置为0，保持正输入不变，引入了非线性和稀疏性

使用ReLU函数可以引入非线性，帮助模型学习复杂的特征表示。使模型不仅能解决线性的问题,还可以解决更复杂的问题.根据万能近似定理,通过非线性激活函数的组合，神经网络可以近似任意复杂函数.

#### CNN背景

假设输入一张 100x100 的单通道图，一个全连接层需要10000个权重才能仅仅让一个输出神经元连接到所有输入像素.作为对比，一个 3x3 的卷积核总共需要9个权重参数？

In [ ]:
import numpy as np

def conv2d(x, w, b, stride=1, padding=0):
    # 对输入进行填充
    x = np.pad(x, ((0,0), (0,0), (padding,padding), (padding,padding)), 
               constant_values=0, mode='constant')
    
    # 获取输入和卷积核的维度信息
    batch_size, in_channels, in_height, in_width = x.shape
    out_channels, kernel_channels, kernel_size, kernel_size = w.shape
    
    # 确保输入通道数与卷积核通道数匹配
    assert in_channels == kernel_channels, "输入通道数与卷积核通道数不匹配"
    
    # 先计算输出特征图的尺寸
    out_height = (in_height - kernel_size) // stride + 1
    out_width = (in_width - kernel_size) // stride + 1

    # 初始化输出
    y = np.zeros((batch_size, out_channels, out_height, out_width))

    for i in range(batch_size):          # 遍历批次
        for c_out in range(out_channels): # 遍历输出通道
            for h in range(out_height):   # 遍历输出高度
                for w_idx in range(out_width): # 遍历输出宽度
                    # 计算窗口起始位置
                    start_h = h * stride
                    start_w = w_idx * stride
                    
                    # 提取滑动窗口区域
                    window = x[i, :, start_h:start_h+kernel_size, 
                              start_w:start_w+kernel_size]
                    #print(np.sum(window * w[c_out]) + b[c_out])
                    y[i, c_out, h, w_idx] = np.sum((np.sum(window * w[c_out]) + b[c_out]))
    
    return y

if __name__ == "__main__":
    # 阶段一: 验证特征检测
    # 1. 定义一个 5x5 的图像，中心有一个“十字”图案
    image_centered = np.array([[
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0]
    ]], dtype=np.float32).reshape(1, 1, 5, 5)  # (batch, channels, height, width)
    
    # 2. 设计一个 3x3 的卷积核，检测十字图案
    w = np.array([[[
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]
    ]]], dtype=np.float32).reshape(1, 1, 3, 3)  # (out_channels, in_channels, kernel_size, kernel_size)

    b = np.array([0], dtype=np.float32)  # 偏置
    
    # 3. 执行卷积，观察输出
    res = conv2d(image_centered, w, b, padding=1, stride=1)
    print("阶段一输出（中心十字）:")
    #print(res.shape)
    print(res[0, 0]) 
    
    # 阶段二: 平移不变性
    # 1. 创建一个新图像，将“十字”图案向右下方平移一格
    image_shifted = np.array([[
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1],
        [0, 0, 0, 1, 0]
    ]], dtype=np.float32).reshape(1, 1, 5, 5)
    
    # 2. 使用完全相同的卷积核进行卷积，并观察输出
    res_shifted = conv2d(image_shifted, w, b, padding=1, stride=1)
    print("\n阶段二输出（平移后十字）:")
    print(res_shifted[0, 0])  # 输出第一个样本、第一个通道的特征图


In [ ]:
from mpmath import ker
import numpy as np

# 创建4x4特征图，左上角有一个2x2的高激活区域
feature_map1 = [[11, 2, 1, 3],
                [10, 3, 1, 1],
                [1, 1, 2, 1],
                [3, 1, 1, 1]]
feature_map1 = np.array(feature_map1)
# feature_map1[:2, :2] = 10  # 2x2高激活区域
# feature_map1[feature_map1 == 0] = 1  # 其他区域设为低激活值


feature_map2 = [[3, 11, 3, 1],
                [1, 10, 3, 1],
                [1, 1, 1, 2],
                [1, 3, 1, 1]]
feature_map2 = np.array(feature_map2)
# feature_map2[1:3, 0:2] = 10  # 平移后的2x2高激活区域
# feature_map2[feature_map2 == 0] = 1  # 其他区域设为低激活值

feature_map1 = feature_map1.reshape(1, 1, 4, 4)
feature_map2 = feature_map2.reshape(1, 1, 4, 4)

print("第一个特征图（原始激活模式）:")
print(feature_map1)
print("\n第二个特征图（激活模式平移1个像素）:")
print(feature_map2)

def max_pool2d(x, kernel_size=2, stride=2):
    in_batch,in_channel,in_height,in_width = x.shape
    out_width,out_height = (int)(in_width/kernel_size),(int)(in_height/kernel_size)
    
    y = np.zeros((in_batch,in_channel,out_height,out_width))
    for i in range(in_batch):
        for j in range(in_channel):
            for k in range(out_height):
                for l in range(out_width):
                    y[i,j,k,l] = np.max(x[i,j,k*stride:k*stride+kernel_size,l*stride:l*stride+kernel_size])
    return y
res1 = max_pool2d(feature_map1)
print(res1)
res2 = max_pool2d(feature_map2)
print(res2)

最大池化有“聚焦局部最大值，忽略位置细节”的特性,“平移不变性” 的核心定义是：当输入特征的位置发生微小移动时，输出特征仍能保留关键信息,由于实验现象,可以得出微小平移不改变 “关键激活信  息"故证明了最大池化层能提供一定程度的“平移不变性”

池化层的核心功能之一是主动降低特征图的空间维度 高 x 宽 ,提取其中关键的特征，从而从 “计算量”“内存占用”“过拟合风险” 三个维度提升网络效率


如果换成平均池化,输出的特征图会有很明显的平移变化,平均池化对平移更敏感，且弱化高激活特,导致高激活特征的对比度大幅下降.

最大池化	突出局部关键特征 适合需要强化显著特征的场景

平均池化	保留局部全局信息 适合需要平滑输出、保留整体趋势的场景

softmax 函数的输出向量具备以下数学特性：
1. 每个元素都在 0 到 1 之间，且所有元素的和为 1。
2. 输出向量的每个元素都可以被理解为类别的概率。
3. 输出向量的元素之间是互斥的，即一个元素的高值会抑制其他元素的高值。
4. 扩大了高值元素和低值元素的差距，使得高值元素的概率差异更大。
故而使得softmax函数在分类中可以扩大高值元素的概率差异，从而提高分类的准确性。并可以直接计算出各个分类类型的概率.

In [ ]:
def Softmax(x,axis):
    """
    计算softmax函数
    :param x: 输入向量
    :return: 输出向量
    """
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=axis)

In [ ]:
import numpy as np
import gzip
import os
import struct
from array import array

# (在此之前应有已实现的 conv2d, relu, max_pool2d, flatten,linear_layer函数)
# 固定随机种子，保证权重初始化一致
np.random.seed(114514)

def softmax(x):
    #axis = logits.ndim - 1
    """
    实现Softmax函数。
    """
    # ===== 在此实现 =====
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# --- MNIST 数据集读取函数 ---
def read_images(filename):
    """
    读取MNIST图像文件
    参数:
      filename: MNIST图像文件路径
    返回:
      images: 图像数组列表
    """
    with open(filename, 'rb') as file:
        magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        if magic != 2051:
            raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
        
        image_data = array("B", file.read())
        
    images = []
    for i in range(size):
        img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
        img = img.reshape(rows, cols)
        images.append(img)
    
    return images

# =========================================================
# ===== 任务：请根据下面的规约，在此处实现 TinyCNN_for_MNIST 类 =====
# =========================================================
#
# --- 模型架构规约 ---
# 1. 构造函数 `__init__(self)`:
# 架构固定：Conv(1->4, k=3, stride=1, pad=1) -> ReLU -> MaxPool(2x2, s=2) -> Flatten -> Linear(->10 类)
# 2. 前向传播方法 `forward(self, x)`:
#    - 接收一个形状为 (N, 1, 28, 28) 的张量 x。
#    - 按照以下顺序依次调用你实现的算子：
#      Conv2d -> ReLU -> MaxPool2d -> Flatten -> Linear -> Softmax
#    - 返回最终的 logits (Linear层输出) 和 probs (Softmax层输出)。
def Linear(x, output):
    # 修正权重维度：(in_features, out_features) → 确保 x·w 维度匹配
    w = np.random.randn(x.shape[1], output)
    b = np.zeros(output)
    return np.dot(x, w) + b
class TinyCNN_for_MNIST:
# ===== 在此实现你的类 =====
    def __init__(self):
        #self.conv1 = self.conv_(x, 4, 3, 1, 1)
        #self.relu1 = relu()
        #self.maxpool1 = max_pool2d(2, 2)
        #self.flatten = flatten()
        #self.linear1 = Linear()
        #self.softmax = Softmax()
        pass
    def conv_ (self,x, out_, k, s, p):
        res=np.zeros((out_,x.shape[0],x.shape[2],x.shape[3]))
        for i in range(out_):
            w = np.random.randn(x.shape[1], k, k).reshape(1, x.shape[1], k, k)
            b = np.zeros(out_).reshape(1, out_, 1, 1)
            res[i] = conv2d(x, w, b, s, p)
        return res
    def forward(self, x):

        x = self.conv_(x, 4, 3, 1, 1)
        x = relu(x)
        x = max_pool2d(x,2,2)
        x = flatten(x)
        x = Linear(x,10)
        #print(x)
        x2 = softmax(x)
        return x,x2
    

# --- 测试脚本 ---
if __name__ == "__main__":
    # 1. 设置 MNIST 测试集文件路径
    # !! 请将此路径修改为你自己的文件路径
    mnist_test_file = "E:\\download\\AI派2025招新第一轮测试\\AI派2025招新第一轮测试\\mnist\\train-images.idx3-ubyte"

    if not os.path.exists(mnist_test_file):
        print(f"错误：找不到 MNIST 测试集文件 '{mnist_test_file}'")
    else:
        # 2. 加载所有测试图像
        test_images = read_images(mnist_test_file)
        # 3. 选取第一张图像作为测试输入
        first_test_image = test_images[0]
        # 4. 预处理图像
        input_tensor = (first_test_image.astype(np.float32) / 255.0 - 0.5) * 2.0
        input_tensor = np.expand_dims(input_tensor, axis=(0, 1))
        # 5. 实例化模型并执行前向传播
        model = TinyCNN_for_MNIST()
        logits, probs = model.forward(input_tensor)

        print("Input Tensor Shape:", input_tensor.shape)
        print("Logits shape:", logits.shape, "Probs shape:", probs.shape)
        np.set_printoptions(precision=8, suppress=False)
        print("\nLogits:", logits[0])
        print("Probs:", probs[0])
        print("\nChecksum logits sum:", float(np.sum(logits)))
        print("Checksum probs sum:", float(np.sum(probs)))

In [ ]:
# import numpy as np
# import gzip
# import os
# import struct
# from array import array

# # 网络层实现（保持不变）
# class Conv2d:
#     def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.kernel_size = kernel_size
#         self.stride = stride
#         self.padding = padding
#         self.weights = np.random.randn(out_channels, in_channels, kernel_size, kernel_size) * np.sqrt(2.0 / (in_channels * kernel_size * kernel_size))
#         self.bias = np.zeros(out_channels)
    
#     def __call__(self, x):
#         N, C_in, H, W = x.shape
#         K, S, P = self.kernel_size, self.stride, self.padding
#         H_out = (H + 2*P - K) // S + 1
#         W_out = (W + 2*P - K) // S + 1
#         x_padded = np.pad(x, ((0,0), (0,0), (P,P), (P,P)), mode='constant')
#         output = np.zeros((N, self.out_channels, H_out, W_out))
        
#         for n in range(N):
#             for c_out in range(self.out_channels):
#                 for h in range(H_out):
#                     for w in range(W_out):
#                         h_start, h_end = h * S, h * S + K
#                         w_start, w_end = w * S, w * S + K
#                         window = x_padded[n, :, h_start:h_end, w_start:w_end]
#                         output[n, c_out, h, w] = np.sum(window * self.weights[c_out]) + self.bias[c_out]
#         return output

# class ReLU:
#     def __call__(self, x):
#         return np.maximum(0, x)

# class MaxPool2d:
#     def __init__(self, kernel_size, stride):
#         self.kernel_size = kernel_size
#         self.stride = stride
    
#     def __call__(self, x):
#         N, C, H, W = x.shape
#         K, S = self.kernel_size, self.stride
#         H_out = (H - K) // S + 1
#         W_out = (W - K) // S + 1
#         output = np.zeros((N, C, H_out, W_out))
        
#         for n in range(N):
#             for c in range(C):
#                 for h in range(H_out):
#                     for w in range(W_out):
#                         h_start, h_end = h * S, h * S + K
#                         w_start, w_end = w * S, w * S + K
#                         output[n, c, h, w] = np.max(x[n, c, h_start:h_end, w_start:w_end])
#         return output

# class Flatten:
#     def __call__(self, x):
#         return x.reshape(x.shape[0], -1)

# class Linear:
#     def __init__(self, in_features, out_features):
#         self.weights = np.random.randn(out_features, in_features) * np.sqrt(1.0 / in_features)
#         self.bias = np.zeros(out_features)
    
#     def __call__(self, x):
#         return np.dot(x, self.weights.T) + self.bias

# # Softmax函数
# np.random.seed(114514)
# def softmax(logits):
#     max_val = np.max(logits, axis=1, keepdims=True)
#     exp_x = np.exp(logits - max_val)
#     return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# ## ---------------------- 修复后的MNIST读取函数（自动适配压缩/非压缩） ----------------------
# def read_images(filename):
#     """
#     读取MNIST图像文件（自动适配.gz压缩和非压缩格式）
#     参数：filename - 图像文件路径（如 train-images-idx3-ubyte 或 train-images-idx3-ubyte.gz）
#     返回：images - 图像列表，每个元素是(28,28)的numpy数组
#     """
#     # 关键：根据文件名是否以.gz结尾，选择打开方式
#     if filename.endswith('.gz'):
#         # 压缩文件：用gzip.open读取
#         import gzip
#         open_func = gzip.open
#     else:
#         # 非压缩文件：用普通open读取
#         open_func = open
    
#     # 用选择的方式打开文件
#     with open_func(filename, 'rb') as file:
#         # 读取MNIST文件头（前16字节：魔术数、图像数、高、宽）
#         magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
        
#         # 校验文件类型（避免误读标签文件）
#         if magic == 2049:
#             raise ValueError("错误：你读取的是MNIST标签文件（魔术数2049）！请替换为图像文件（魔术数2051），图像文件名含'images'（如train-images-idx3-ubyte）")
#         if magic != 2051:
#             raise ValueError(f"错误：未知文件类型！魔术数为{magic}，MNIST图像文件魔术数应为2051")
        
#         # 读取所有像素数据（无符号字节类型，范围0-255）
#         from array import array
#         image_data = array("B", file.read())
    
#     # 将字节数据转换为(28,28)的numpy数组列表
#     images = []
#     for i in range(size):
#         # 提取第i张图像的字节（每张图有rows*cols=784个字节）
#         img_flat = image_data[i * rows * cols : (i+1) * rows * cols]
#         # 重塑为(28,28)的二维数组
#         img = np.array(img_flat).reshape(rows, cols)
#         images.append(img)
    
#     return images
    

# # 模型类
# class TinyCNN_for_MNIST:
#     def __init__(self):
#         self.conv1 = Conv2d(1, 4, 3, 1, 1)
#         self.relu1 = ReLU()
#         self.maxpool1 = MaxPool2d(2, 2)
#         self.flatten = Flatten()
#         self.linear1 = Linear(4*14*14, 10)
        
#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu1(x)
#         x = self.maxpool1(x)
#         x = self.flatten(x)
#         logits = self.linear1(x)
#         probs = softmax(logits)
#         return logits, probs

# # 测试脚本
# if __name__ == "__main__":
#     # 确保使用正确的图像文件，通常图像文件名为：
#     # 训练集：train-images-idx3-ubyte.gz
#     # 测试集：t10k-images-idx3-ubyte.gz
#     mnist_test_file = "E:\\download\\AI派2025招新第一轮测试\\AI派2025招新第一轮测试\\mnist\\train-images.idx3-ubyte"

#     if not os.path.exists(mnist_test_file):
#         print(f"错误：找不到文件 '{mnist_test_file}'")
#         print("请检查路径是否正确，确保文件名包含 'images' 而非 'labels'")
#     else:
#         try:
#             test_images = read_images(mnist_test_file)
#             print(f"成功读取 {len(test_images)} 张图像")
            
#             first_test_image = test_images[0]
#             input_tensor = (first_test_image.astype(np.float32) / 255.0 - 0.5) * 2.0
#             input_tensor = np.expand_dims(input_tensor, axis=(0, 1))
            
#             model = TinyCNN_for_MNIST()
#             logits, probs = model.forward(input_tensor)

#             print("Input Tensor Shape:", input_tensor.shape)
#             print("Logits shape:", logits.shape, "Probs shape:", probs.shape)
#             print("\nLogits:", logits[0])
#             print("Probs:", probs[0])
#             print("\nProbs sum:", float(np.sum(probs)))  # 应为1.0左右
#         except ValueError as e:
#             print(f"读取失败：{e}")
#             print("请确认使用的是图像文件，MNIST文件命名规则：")
#             print("- 图像文件：*images-idx3-ubyte.gz")
#             print("- 标签文件：*labels-idx1-ubyte.gz")


### 注意力

词嵌入是将文本中的词语或子词转换为低维稠密向量的技术,将每个词语映射到一个连续的向量空间中,使得词语之间的语义关系可以通过向量之间的距离来表示。
作用在于让计算机能够理解词语的语义，并用数学方式表示词语之间的关系。

传统的词表示方法容易出现维度爆炸,即词数过多导致维度过大,和语义缺失的问题,例如one_hot编码,词向量之间点积为零,无法数学方法表达词义之间的距离

词嵌入的局限性有 
- 容易陷入一词多意的困境和领域偏移的问题, 
- 且对大量的优质训练数据有依赖, 
- 缺乏可解释性等等

一种常见的词嵌入模型是 Word2Vec 由 Google 在 2013 年提出,Word2Vec 的核心思想是 “上下文相似的词，其嵌入向量嵌入也相似
- 低维稠密表示
相比 One-Hot 编码，Word2Vec 的向量维度固定 50~300 维，极大降低了计算成本，同时保留语义信息。
- 捕捉语义和语法关联
语义上：“猫” 和 “狗” 的向量距离近，“医生” 和 “医院” 距离近；
语法上：“run” 和 “ran”、“big” 和 “bigger”的向量关系相似。
- 训练高效
引入 “负采样” 即 Negative Sampling 和 “层次 Softmax” 技术，避免传统神经网络对所有词汇的概率计算，使训练速度提升 10~100 倍，可处理数十亿级文本。
- 静态词向量
每个词对应唯一向量,如 “苹果” 在 “吃苹果” 和 “苹果公司” 中向量相同,无法处理一词多义.这是其主要局限。


多头多头自注意力核心是通过多个并行 “注意力头”，从不同视角学习序列内关联，再融合结果提升表达能力。每个头聚焦不同关联类型（如语法、语义），最后拼接并线性映射输出。
缩放点积注意力公式：Attention (Q,K,V) = Softmax ((QKᵀ)/√dₖ) V 。
Q（查询）：需关注的对象；K（键）：供关注的特征，Q 与 K 的点积算相似度；V（值）：实际需加权的信息。
√dₖ（缩放因子）：避免 dₖ过大导致点积结果过大，使 Softmax 梯度稳定。
Softmax：将相似度归一化为权重，最终输出是 V 的加权和。
带掩码时加 Mask 项，屏蔽无效信息（如 padding 或未来内容）。

#### 缩放点积注意力（Scaled Dot-Product Attention）计算公式

#### 形式
$$
\text{Attention}(Q, K, V) = \text{Softmax}\left( \frac{QK^T}{\sqrt{d_k}} \right) V
$$


### 参数含义说明

| 参数符号         | 含义解释                                                                 |
|------------------|--------------------------------------------------------------------------|
| $Q$              | 查询向量（Query），代表当前需要关注的对象，用于计算与其他元素的关联强度   |
| $K$              | 键向量（Key），代表可供关注的特征，与$Q$的点积结果反映两者关联程度       |
| $V$              | 值向量（Value），代表实际需加权的信息，最终输出是$V$基于注意力权重的加权和 |
| $QK^T$           | $Q$与$K$的转置矩阵相乘，得到原始关联度矩阵，元素$(i,j)$表示第$i$个查询与第$j$个键的关联强度 |
| $\sqrt{d_k}$     | 缩放因子，$d_k$为$Q$和$K$的维度，避免维度过大导致点积结果过大，缓解Softmax梯度消失 |
| $\text{Softmax}$ | 归一化函数，将关联度转换为注意力权重，使每个查询对应的权重之和为1         |


In [ ]:
import numpy as np
#from numpy import softmax
np.random.seed(114514)

def scaled_dot_product_attention(Q, K, V, mask=None):
    d_k = Q.shape[-1]
    scores = np.matmul(Q, K.transpose(0, 1, 3, 2)) / np.sqrt(d_k)
    if mask is not None:
        scores += mask * -1e9
    attention_weights = softmax(scores)
    output = np.matmul(attention_weights, V)
    return output, attention_weights

def multi_head_attention(embed_size, num_heads, input, mask=None):
    head_size = embed_size // num_heads
    wQ = np.random.randn(batch_size, seq_len, num_heads, head_size)
    wK = np.random.randn(batch_size, seq_len, num_heads, head_size)
    wV = np.random.randn(batch_size, seq_len, num_heads, head_size)
    wO = np.random.randn(batch_size, seq_len, num_heads, head_size)
    output, attention_weights = scaled_dot_product_attention(wQ, wK, wV, mask)
    return output, attention_weights

if __name__ == "__main__":
    batch_size = 10
    seq_len = 20
    embed_size = 128
    num_heads = 8
    input = np.random.randn(batch_size, seq_len, embed_size) 
    output, weights = multi_head_attention(embed_size, num_heads, input)
	
    print(output.shape, weights.shape)
    print(output[0][0][:10], weights[0][0][0][:10])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    """基于PyTorch的多头自注意力实现"""
    def __init__(self, embed_size, num_heads, dropout=0.1):
        super().__init__()
        assert embed_size % num_heads == 0, "嵌入维度(embed_size)必须能被头数(num_heads)整除"

        self.embed_size = embed_size  # 总嵌入维度
        self.num_heads = num_heads    # 注意力头数量
        self.head_size = embed_size // num_heads  # 单个头的维度

        self.w_q = nn.Linear(embed_size, embed_size)
        self.w_k = nn.Linear(embed_size, embed_size)
        self.w_v = nn.Linear(embed_size, embed_size)
        self.w_o = nn.Linear(embed_size, embed_size)
        self.dropout = nn.Dropout(dropout)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        d_k = Q.size(-1)  # 单个头的维度（用于缩放）

        scores = torch.matmul(Q, K.transpose(-2, -1))  # 形状 batch, heads, seq_q, seq_k
        scores = scores / torch.sqrt(torch.tensor(d_k, dtype=torch.float32, device=scores.device))
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        attn_weights = F.softmax(scores, dim=-1)  
        attn_weights = self.dropout(attn_weights)

        output = torch.matmul(attn_weights, V)  
        
        return output, attn_weights

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)  # 获取批次大小

        Q = self.w_q(query) 
        K = self.w_k(key)   
        V = self.w_v(value) 
        
        Q = Q.view(batch_size, -1, self.num_heads, self.head_size).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.head_size).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.head_size).transpose(1, 2)

        attn_output, attn_weights = self.scaled_dot_product_attention(Q, K, V, mask)
        attn_output =attn_output.transpose(1, 2).contiguous() 
        attn_output =attn_output.view(batch_size, -1, self.embed_size)  # 拼接为embed_size

        final_output = self.w_o(attn_output)
        
        return final_output, attn_weights

# 测试代码
if __name__ == "__main__":
    # 1. 构造测试输入（与任务2保持一致的形状）
    batch_size = 10
    seq_len = 20
    embed_size = 128
    num_heads = 8
    input_tensor = torch.randn(batch_size, seq_len, embed_size)  # 随机输入张量

    model = MultiHeadAttention(embed_size, num_heads)

    output, attn_weights = model(input_tensor, input_tensor, input_tensor)

    print(f"输出张量形状: {output.shape}")  
    print(f"注意力权重形状: {attn_weights.shape}") 
    print("\n输出张量[0,0,:10]（第1个样本、第1个位置的前10个维度）:")
    print(output[0][0][:10])
    print("\n注意力权重[0,0,0,:10]（第1个样本、第1个头部、第1个位置的前10个权重）:")
    print(attn_weights[0][0][0][:10])

### 库调用

In [ ]:
from transformers import pipeline
import requests
from PIL import Image
# load pipeline
ckpt = "google/siglip2-base-patch16-224"
image_classifier = pipeline(model=ckpt, task="zero-shot-image-classification")

# load image and candidate labels
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
candidate_labels = ["2 cats", "a plane", "a remote"]
image = Image.open(requests.get(url, stream=True).raw)
# run inference
outputs = image_classifier(image, candidate_labels)
print(outputs)

#### 结果
![结果](res.png)


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoProcessor, AutoModel
from PIL import Image
import numpy as np

# 1. 配置参数
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"  # 国内加速
MODEL_CHECKPOINT = "google/siglip2-base-patch16-224"
DATASET = "ethz/food101"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32  # 根据显存调整
TOP_K = 5  # 计算top-5准确率

# 2. 加载数据集（只取验证集，每类前10张）
dataset = load_dataset(DATASET, split="validation")

# 获取标签ID到文本的映射（从数据集README提取的101类食物名称）
# 完整标签列表见：https://huggingface.co/datasets/ethz/food101#labels
food_labels = [
    "apple pie", "baby back ribs", "baklava", "beef carpaccio", "beef tartare",
    "beet salad", "beignets", "bibimbap", "bread pudding", "breakfast burrito",
    "bruschetta", "caesar salad", "cannoli", "caprese salad", "carrot cake",
    "ceviche", "cheesecake", "cheese plate", "chicken curry", "chicken quesadilla",
    "chicken wings", "chocolate cake", "chocolate mousse", "churros", "clam chowder",
    "club sandwich", "crab cakes", "creme brulee", "croque madame", "cup cakes",
    "deviled eggs", "donuts", "dumplings", "edamame", "eggs benedict", "escargots",
    "falafel", "filet mignon", "fish and chips", "foie gras", "french fries",
    "french onion soup", "french toast", "fried calamari", "fried rice", "frozen yogurt",
    "garlic bread", "gnocchi", "greek salad", "grilled cheese sandwich", "grilled salmon",
    "guacamole", "gyoza", "hamburger", "hot and sour soup", "hot dog", "huevos rancheros",
    "hummus", "ice cream", "lasagna", "lobster bisque", "lobster roll sandwich",
    "macaroni and cheese", "macarons", "miso soup", "mussels", "nachos", "omelette",
    "onion rings", "oysters", "pad thai", "paella", "pancakes", "panna cotta", "peking duck",
    "pho", "pizza", "pork chop", "poutine", "prime rib", "pulled pork sandwich", "ramen",
    "ravioli", "red velvet cake", "risotto", "samosa", "sashimi", "scallops", "seaweed salad",
    "shrimp and grits", "spaghetti bolognese", "spaghetti carbonara", "spring rolls",
    "steak", "strawberry shortcake", "sushi", "tacos", "takoyaki", "tiramisu", "tuna tartare",
    "waffles"
]

# 筛选每类前10张图（共101×10=1010张）
def filter_per_class(example, class_id, count_per_class=10):
    """保留每个类别的前count_per_class张样本"""
    if example["label"] not in class_id:
        class_id[example["label"]] = 0
    if class_id[example["label"]] < count_per_class:
        class_id[example["label"]] += 1
        return True
    return False

class_counter = {}
filtered_dataset = dataset.filter(
    filter_per_class,
    fn_kwargs={"class_id": class_counter, "count_per_class": 10}
)
print(f"筛选后样本数: {len(filtered_dataset)}")  # 应输出1010


# 3. 加载模型和处理器
processor = AutoProcessor.from_pretrained(MODEL_CHECKPOINT)
model = AutoModel.from_pretrained(
    MODEL_CHECKPOINT,
    torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32,
    device_map="auto"
)
model.eval()  # 推理模式


# 4. 批量推理并计算top-5准确率
correct = 0
total = 0

# 生成候选文本（统一格式为"a photo of {food}"，符合SigLIP训练习惯）
candidate_texts = [f"a photo of {label}" for label in food_labels]

# 按批次处理
for i in range(0, len(filtered_dataset), BATCH_SIZE):
    batch = filtered_dataset[i:i+BATCH_SIZE]
    images = [Image.fromarray(img).convert("RGB") for img in batch["image"]]
    true_labels = batch["label"]  # 真实标签ID（0-100）
    
    # 预处理
    inputs = processor(
        text=candidate_texts,
        images=images,
        padding="max_length",
        max_length=64,
        return_tensors="pt"
    ).to(DEVICE, dtype=torch.float16 if DEVICE == "cuda" else torch.float32)
    
    # 推理（关闭梯度计算）
    with torch.no_grad():
        outputs = model(** inputs)
    
    # 解析结果：获取每个样本的top-5预测标签ID
    logits = outputs.logits_per_image  # 形状：[batch_size, 101]
    top5_preds = torch.topk(logits, k=TOP_K, dim=1).indices.cpu().numpy()  # [batch_size, 5]
    
    # 统计正确数（真实标签是否在top-5中）
    for pred_ids, true_id in zip(top5_preds, true_labels):
        if true_id in pred_ids:
            correct += 1
    total += len(batch)
    
    # 打印进度
    if (i // BATCH_SIZE) % 5 == 0:
        print(f"已处理 {i+len(batch)}/{len(filtered_dataset)} 样本，当前准确率: {correct/total:.4f}")

# 输出最终结果
print(f"\nTop-{TOP_K} 准确率: {correct/total:.4f} ({correct}/{total})")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModel
import torch
import umap  # 修改导入方式
from tqdm import tqdm

# 设置中文字体
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]
plt.rcParams["axes.unicode_minus"] = False  # 正确显示负号

# 定义要处理的类别和数量
CLASSES = ["pizza", "sushi", "hamburger", "ice_cream", "dumplings"]
NUM_PER_CLASS = 100

# 加载Food101数据集（只加载训练集）
print("加载Food101数据集...")
dataset = load_dataset("food101", split="train")

# 筛选出我们需要的类别和样本
filtered_samples = []
labels = []

print("筛选数据集中的样本...")
for cls in CLASSES:
    # 找到该类别的所有样本
    class_samples = [sample for sample in dataset if sample["label"] == dataset.features["label"].str2int(cls)]
    # 只取前NUM_PER_CLASS个样本
    selected_samples = class_samples[:NUM_PER_CLASS]
    filtered_samples.extend(selected_samples)
    labels.extend([cls] * len(selected_samples))

print(f"共筛选出 {len(filtered_samples)} 个样本")

# 加载SigLIP模型和图像处理器
print("加载SigLIP模型...")
model_name = "google/siglip-base-patch16-224"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 将模型移动到可用设备
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# 提取图像特征
print("提取图像特征...")
embeddings = []

# 批量处理图像以提高效率
batch_size = 32
for i in tqdm(range(0, len(filtered_samples), batch_size)):
    batch_samples = filtered_samples[i:i+batch_size]
    
    # 处理图像
    inputs = processor(
        images=[sample["image"] for sample in batch_samples],
        return_tensors="pt",
        padding=True,
        truncation=True
    )
    
    # 移动到设备并获取特征
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(** inputs)
    
    # 使用[CLS] token的输出作为图像嵌入
    batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    embeddings.extend(batch_embeddings)

# 转换为numpy数组
embeddings = np.array(embeddings)
print(f"提取的特征形状: {embeddings.shape}")

# 使用UMAP降维到2维（修改UMAP使用方式）
print("使用UMAP进行降维...")
umap_model = umap.UMAP(n_components=2, random_state=42)  # 这里是修改的关键
embeddings_2d = umap_model.fit_transform(embeddings)

# 可视化结果
print("绘制可视化结果...")
plt.figure(figsize=(12, 10))
scatter = sns.scatterplot(
    x=embeddings_2d[:, 0], 
    y=embeddings_2d[:, 1],
    hue=labels,
    palette=sns.color_palette("husl", len(CLASSES)),
    s=50,
    alpha=0.7
)

plt.title("SigLIP特征的UMAP降维可视化 (Food101的5个类别)", fontsize=16)
plt.xlabel("UMAP维度 1", fontsize=12)
plt.ylabel("UMAP维度 2", fontsize=12)
plt.legend(title="食物类别", title_fontsize=12, fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()

# 保存图像
plt.savefig("food101_siglip_umap.png", dpi=300, bbox_inches="tight")
print("可视化结果已保存为 food101_siglip_umap.png")

# 显示图像
plt.show()
    

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModel
from tqdm import tqdm
import os

# 设置中文字体
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]
plt.rcParams["axes.unicode_minus"] = False

# 配置参数
CLASSES = ["pizza", "sushi", "hamburger", "ice_cream", "dumplings"]
TRAIN_NUM_PER_CLASS = 1  # 每类使用1张图片训练
TEST_NUM_PER_CLASS = 99  # 每类使用99张图片测试
EPOCHS = 100
BATCH_SIZE = 16
LR = 0.001
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LOG_DIR = "./runs/siglip_linear_probing"

# 创建TensorBoard记录器
writer = SummaryWriter(LOG_DIR)

# 1. 加载数据集和预处理
print("加载数据集...")
# 如果之前已下载数据集，可使用load_from_disk加载本地数据
try:
    dataset = load_dataset("food101", split="train")
except:
    # 从本地加载（请替换为你的实际路径）
    dataset = load_from_disk("./food101/train")

# 分割训练集和测试集
train_samples = []
test_samples = []
train_labels = []
test_labels = []

for cls in CLASSES:
    class_idx = dataset.features["label"].str2int(cls)
    class_samples = [s for s in dataset if s["label"] == class_idx]
    
    # 每类第一张作为训练样本，其余作为测试样本
    train_samples.append(class_samples[0])
    train_labels.append(cls)
    test_samples.extend(class_samples[1:TEST_NUM_PER_CLASS+1])
    test_labels.extend([cls] * TEST_NUM_PER_CLASS)

print(f"训练集: {len(train_samples)}个样本")
print(f"测试集: {len(test_samples)}个样本")

# 2. 加载SigLIP模型并提取特征
print("加载SigLIP模型并提取特征...")
model_name = "google/siglip-base-patch16-224"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(DEVICE)
model.eval()

# 提取特征的函数
def extract_features(samples):
    embeddings = []
    with torch.no_grad():
        for sample in tqdm(samples, desc="提取特征"):
            inputs = processor(images=sample["image"], return_tensors="pt").to(DEVICE)
            outputs = model(**inputs)
            # 使用[CLS] token的输出作为特征
            embeddings.append(outputs.last_hidden_state[:, 0, :].cpu().numpy()[0])
    return np.array(embeddings)

# 提取训练集和测试集的特征
X_train = extract_features(train_samples)
X_test = extract_features(test_samples)

# 标签编码
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
y_test = label_encoder.transform(test_labels)

# 3. 创建数据集和数据加载器
class FeatureDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

train_dataset = FeatureDataset(X_train, y_train)
test_dataset = FeatureDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# 4. 定义线性探针模型
class LinearProbe(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        return self.fc(x)

# 初始化模型、损失函数和优化器
input_dim = X_train.shape[1]
num_classes = len(CLASSES)
model_probe = LinearProbe(input_dim, num_classes).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_probe.parameters(), lr=LR)

# 5. 训练模型
print("开始训练线性探针...")
best_acc = 0.0

for epoch in range(EPOCHS):
    model_probe.train()
    train_loss = 0.0
    train_preds = []
    train_true = []
    
    for features, labels in train_loader:
        features, labels = features.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model_probe(features)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * features.size(0)
        _, preds = torch.max(outputs, 1)
        train_preds.extend(preds.cpu().numpy())
        train_true.extend(labels.cpu().numpy())
    
    # 计算训练集指标
    train_loss /= len(train_dataset)
    train_acc = accuracy_score(train_true, train_preds)
    
    # 在测试集上评估
    model_probe.eval()
    test_loss = 0.0
    test_preds = []
    test_true = []
    
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            outputs = model_probe(features)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item() * features.size(0)
            _, preds = torch.max(outputs, 1)
            test_preds.extend(preds.cpu().numpy())
            test_true.extend(labels.cpu().numpy())
    
    # 计算测试集指标
    test_loss /= len(test_dataset)
    test_acc = accuracy_score(test_true, test_preds)
    
    # 记录到TensorBoard
    writer.add_scalar('Loss/Train', train_loss, epoch)
    writer.add_scalar('Loss/Test', test_loss, epoch)
    writer.add_scalar('Accuracy/Train', train_acc, epoch)
    writer.add_scalar('Accuracy/Test', test_acc, epoch)
    
    # 保存最佳模型
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model_probe.state_dict(), "best_linear_probe.pth")
    
    # 打印进度
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
        print("-" * 50)

writer.close()
print(f"最佳测试准确率: {best_acc:.4f}")

# 6. 绘制训练曲线
# 从TensorBoard日志中读取数据并绘图
# 这里简化处理，实际中可以使用tensorboard或读取事件文件
print("请使用以下命令查看训练曲线:")
print(f"tensorboard --logdir={LOG_DIR}")

# 绘制简单的训练曲线（需要手动记录数据或从事件文件读取）
# 这里仅作示例框架
plt.figure(figsize=(12, 5))

# 假设我们有记录的loss和acc数据
# 实际使用时需要从TensorBoard日志中提取
epochs_range = range(1, EPOCHS+1)

plt.subplot(1, 2, 1)
plt.plot(epochs_range, [0]*EPOCHS, label='训练损失')  # 这里替换为实际数据
plt.plot(epochs_range, [0]*EPOCHS, label='测试损失')  # 这里替换为实际数据
plt.title('损失曲线')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, [0]*EPOCHS, label='训练准确率')  # 这里替换为实际数据
plt.plot(epochs_range, [0]*EPOCHS, label='测试准确率')  # 这里替换为实际数据
plt.title('准确率曲线')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.savefig('training_curves.png')
plt.show()
    

In [ ]:
import os
import asyncio
import aiohttp
import base64
from PIL import Image
import io
import json
from datasets import load_dataset
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# 配置API环境变量（确保与提供的密钥和Base URL一致）
os.environ["OPENAI_API_KEY"] = "sk-yxhm7vIgkeffD0FU1bE5F797B654482d94CbB6DbBa556b96"
os.environ["OPENAI_BASE_URL"] = "https://api.ai-gaochao.cn/v1"

# 分类类别（与任务2保持一致）
CLASSES = ["pizza", "sushi", "hamburger", "ice_cream", "dumplings"]
TEST_NUM_PER_CLASS = 20  # 每类测试样本数量，控制API消耗

# 图像转Base64编码（API要求的图像格式）
def image_to_base64(image):
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")  # 统一保存为PNG，避免编码兼容性问题
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# 构建提示词（严格约束结构化输出，避免解析失败）
def build_prompt():
    return f"""任务：对提供的食物图片进行分类，仅从以下指定类别中选择一个：{', '.join(CLASSES)}。

输出要求：
1. 仅返回JSON格式结果，不添加任何额外文字、解释或注释；
2. JSON必须只包含一个键"category"，其值为上述类别中的一个（严格匹配，不可自定义）；
3. 若无法明确判断，需选择最可能的类别。

示例输出（仅参考格式）：
{{"category": "pizza"}}
"""

# 异步调用API（单个图像分类的基础单元）
async def classify_image(session, image, prompt):
    base64_image = image_to_base64(image)
    
    # 构造API请求体（适配Gemini通过OpenAI兼容接口的调用格式）
    payload = {
        "model": "gemini-2.5-flash",  # 模型名称需与API支持的一致
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},  # 文本指令
                    {
                        "type": "image_url",  # 图像数据
                        "image_url": {"url": f"data:image/png;base64,{base64_image}"}
                    }
                ]
            }
        ],
        "temperature": 0.0,  # 降低随机性，确保分类结果稳定
        "max_tokens": 50      # 限制输出长度，避免冗余内容导致解析失败
    }
    
    try:
        async with session.post(
            url=f"{os.environ['OPENAI_BASE_URL']}/chat/completions",
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}"
            },
            json=payload,
            timeout=25  # 超时时间，避免长期阻塞
        ) as response:
            if response.status != 200:
                # 打印API错误详情，便于排查问题（如密钥无效、限流等）
                error_msg = await response.text()
                print(f"API调用失败：状态码{response.status}，详情：{error_msg}")
                return None
            
            # 解析API响应（严格按JSON格式提取结果）
            data = await response.json()
            content = data['choices'][0]['message']['content'].strip()  # 去除首尾空白字符
            result = json.loads(content)  # 解析JSON
            return result['category'] if 'category' in result else None
    
    except json.JSONDecodeError:
        print(f"解析模型输出失败：模型返回非JSON内容 -> {content}")
        return None
    except Exception as e:
        print(f"分类过程出错：{str(e)}")
        return None

# 异步批量处理（控制并发数，避免API限流）
async def batch_classify(images, labels, prompt, batch_size=5):
    results = []
    # 创建异步会话（复用连接，提升效率）
    async with aiohttp.ClientSession() as session:
        # 按批次处理，每批并发调用API
        for i in tqdm(range(0, len(images), batch_size), desc="分类进度"):
            batch_images = images[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            
            # 创建当前批次的所有异步任务
            tasks = [classify_image(session, img, prompt) for img in batch_images]
            # 等待当前批次所有任务完成（控制并发量）
            batch_results = await asyncio.gather(*tasks)
            
            # 记录每一个样本的预测结果和真实标签
            for pred, true in zip(batch_results, batch_labels):
                results.append({
                    "predicted": pred,
                    "true": true
                })
    
    return results

# 主异步函数（数据准备 + 分类 + 结果计算）
async def main():
    # 1. 加载数据集（优先Hub，失败则本地加载）
    print("加载Food101数据集...")
    try:
        dataset = load_dataset("food101", split="train")
    except Exception as e:
        print(f"从Hugging Face Hub加载失败，尝试本地加载：{str(e)}")
        from datasets import load_from_disk
        # 注意：需将路径替换为你本地Food101数据集的实际解压路径
        dataset = load_from_disk("./food101/train")#没下...网不行,下不来
    
    # 2. 准备测试集
    test_images = []
    test_labels = []
    for cls in CLASSES:
        class_idx = dataset.features["label"].str2int(cls)  # 类别名转索引
        class_samples = [s for s in dataset if s["label"] == class_idx]  # 筛选该类样本
        
        # 确保样本数量足够，避免索引越界
        if len(class_samples) < 10 + TEST_NUM_PER_CLASS:
            print(f"警告：{cls}类样本不足，仅能提供{len(class_samples)-10}个测试样本（需{TEST_NUM_PER_CLASS}个）")
            selected_samples = class_samples[10:]  # 取剩余所有样本
        else:
            selected_samples = class_samples[10:10+TEST_NUM_PER_CLASS]  # 取第11-30张
        
        # 收集图像和标签
        test_images.extend([s["image"] for s in selected_samples])
        test_labels.extend([cls] * len(selected_samples))
    
    # 检查测试集是否为空
    if len(test_images) == 0:
        print("错误：未收集到任何测试样本，请检查数据集路径或类别筛选逻辑")
        return
    print(f"测试集准备完成：共{len(test_images)}个样本（{len(CLASSES)}类，每类约{TEST_NUM_PER_CLASS}个）")
    
    # 3. 构建提示词
    prompt = build_prompt()
    
    # 4. 执行异步批量分类
    print("开始调用Gemini 2.5 Flash API进行分类...")
    results = await batch_classify(test_images, test_labels, prompt, batch_size=5)
    
    # 5. 计算准确率（过滤预测失败的样本）
    # 筛选有效结果（排除预测为None的样本）
    valid_results = [r for r in results if r["predicted"] is not None]
    if len(valid_results) == 0:
        print("错误：所有样本分类失败，无法计算准确率（检查API密钥、Base URL或网络）")
        return
    
    # 提取真实标签和预测标签
    y_true = [r["true"] for r in valid_results]
    y_pred = [r["predicted"] for r in valid_results]
    accuracy = accuracy_score(y_true, y_pred)  # 计算准确率
    
    # 6. 输出结果
    print(f"\n=== 分类结果汇总 ===")
    print(f"总测试样本数：{len(results)}")
    print(f"有效分类样本数：{len(valid_results)}")
    print(f"最终分类准确率：{accuracy:.4f}（正确{sum([p==t for p,t in zip(y_pred, y_true)])}个 / 有效{len(valid_results)}个）")
    
    # 7. 保存结果到JSON文件（便于后续分析）
    with open("gemini_classification_results.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"\n分类详情已保存到：gemini_classification_results.json")
    
    # 8. 与任务2（SigLIP+线性探针）对比分析
    print(f"\n=== 与任务2（SigLIP+线性探针）准确率对比 ===")
    print("1. 通常情况下，Gemini 2.5 Flash准确率更高：")
    print("   - 原因1：Gemini具备视觉-语言联合理解能力，能处理模糊/复杂背景（如披萨与汉堡的芝士、面包区分）；")
    print("   - 原因2：Gemini内置食物类别知识，无需额外训练线性层，减少小样本过拟合；")
    print("   - 原因3：抗干扰能力更强（如冰淇淋融化、饺子摆盘变化对分类影响更小）。")
    print("2. Gemini的劣势：")
    print("   - 依赖API调用，存在延迟（单样本约0.5-2秒）和成本（按token/图像计费）；")
    print("   - 无法本地化部署，大规模数据分类（如万级样本）成本过高；")
    print("   - 受API限流影响，并发量不能无限制提升。")

# -------------------------- 关键修复：启动异步函数 --------------------------

await main()
